In [1]:

import yaml
import os
from glob import glob
import os
import pandas as pd
import json
import pickle
import datetime

import numpy as np
import json


In [2]:
import sys
sys.path.insert(0,"/home/lmeyers/ReID_complete/SCL_reID/utils/")

from pytorch_eval_reid import eval

/home/lmeyers/anaconda3/envs/mlenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-09-18 12:29:25.341567: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-18 12:29:26.083149: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


finished imports
beginning execution


## Reference train set, query test set

In [3]:
directory_of_models_trained = '/home/lmeyers/contrastive_learning_new_training/64_ids_batch1_sample_num_64/checkpoints/'
# directory_for_ymls = './new_configs_reference_train_query_test/'
# directory_for_jsons = './jsons_for_results/'
yml = "/home/lmeyers/contrastive_learning_new_training/64_ids_batch1_sample_num_64/64_ids_batch1_sample_num_64.yml"

gpu_id = '1'
# if(not os.path.exists(directory_for_ymls)):
#     os.mkdir(directory_for_ymls)
    
# if(not os.path.exists(directory_for_jsons)):
#     os.mkdir(directory_for_jsons)


for model_file in glob(directory_of_models_trained+"*.pth"):

#         reference_file = '/home/lmeyers/summer_bee_data_reextract/new_open_max_ids_batch2/summer_bee_dataset_open_train_bee_64_ids_batch2_sample_num_02.csv'
#         test_file = '/home/lmeyers/summer_bee_data_reextract/new_open_max_ids_batch2/summer_bee_dataset_open_train_bee_64_ids_batch2_sample_num_max.csv'
#         # else:
#         #     reference_file = '/home/lmeyers/summer_bee_data_reextract/new_open_max_ids_batch1/summer_bee_dataset_open_train_bee_64_ids_batch1_sample_num_02.csv'
#         #     test_file = '/home/lmeyers/summer_bee_data_reextract/new_open_max_ids_batch1/summer_bee_dataset_open_train_bee_64_ids_batch1_sample_num_max.csv'


#         ##----------------Filter particular runs (if needed)----------
#         #if num_images == 'max':

#         ##---------- create a temp yml to eval all checkpoints---------------
        
    #open config yaml to update experiment params
    with open(yml, 'r') as fo:
        config = yaml.safe_load(fo)
    
    model_name = os.path.basename(model_file)
    run_dir_name = os.path.dirname(model_file)
    num_epochs = model_name[:-4]
    print(num_epochs)
#         #------------ Initilize and make new dir for each training set-----
    train_file = config['data_settings']["datafiles"]["train"]
#         # Saves wandb folder, checkpoints, and outputs to folder named based on run string, which should contain attrs that differntiate run
    run_str = os.path.basename(train_file)[34:-4] #MAY NEED TO MODIFY BASED ON CSV NAME
#         run_dir_name = os.path.join(config['train_settings']['wandb_dir_path'],run_str+'/')
#         if not os.path.exists(run_dir_name):
#             os.mkdir(run_dir_name)
#         split_parts = run_str.rsplit('_', 1) #String parse csv name
#         if len(split_parts) > 1: # Check if there is at least one underscore in the string
#             # Get the substring after the last underscore
#             num_images = split_parts[1]
#             num_ids = split_parts[0][:2]
#         else:
#             # Handle the case where there are no underscores in the string
#             num_images = run_str 
        
#         # Saves wandb folder, checkpoints, and outputs to folder named based on run string, which should contain attrs that differntiate run
    wandb_name =  config['train_settings']['wandb_project_name']+run_str #set wandb_dir_name
        
#         #Update params
#         config['model_settings']['num_labels']= num_ids
#         print('Num labels ',num_ids)

#         #Check if batch size needs to be updated
#         df = pd.read_csv(train_file)
#         if config['data_settings']['batch_size'] > len(df):
#             config['data_settings']['batch_size'] = len(df)
#             print('Updated batch to contain all Data. Size = ',len(df))
        
#         #Check if print_k needs to be updated for small dataset
#         print_k = config['train_settings']['print_k']
#         if print_k > len(df)/config['data_settings']['batch_size']:
#             print_k = len(df)/config['data_settings']['batch_size']
#             config['train_settings']['print_k'] = print_k
#             print('Updating print_k to contain whole epoch. Num_batches =',print_k)
        
    #Testing a differnt num of epochs based on loop
    config['train_settings']['num_epochs'] = num_epochs

#         #updating datafiles
#         config['data_settings']['datafiles']['train']=train_file
#         config['data_settings']['datafiles']['reference']= reference_file

#         #config['data_settings']['datafiles']['train']=train_csv
#         config['data_settings']['datafiles']['test'] = test_file
#         config['data_settings']['datafiles']['valid']= ''
#         config['data_settings']['datafiles']['query']= test_file

    #update Model path
    config['model_settings']['model_path'] = model_file

#         #update pickle_file to prevent being overwritten
    pickle_file = os.path.join(run_dir_name,'results.pkl')
    config['eval_settings']['pickle_file'] = pickle_file
    csv_file = os.path.join(run_dir_name,"results.csv")
    config['eval_settings']['results_file'] = csv_file

#         #update wandb_project_name
#         config['train_settings']['wandb_project_name'] = wandb_name
#         config['train_settings']['wandb_dir_path'] = run_dir_name #this should make a seperate wandb folder for runs

#         #save yml
    new_yml_file = os.path.join(run_dir_name,"checkpoint_eval.yml")
   
    with open(new_yml_file, 'w') as fo:
            yaml.dump(config,fo)   
        

    #---------- actually run eval--------------
    # !python pytorch_train_and_eval_reid_2.py --config_file {new_yml_file}
    eval(new_yml_file)

#         # Save model to wandb file location to prevent overwriting
#         # new dir in wandb/ will be generated each training run
#         !cp {config['model_settings']['model_path']} {config['train_settings']['wandb_dir_path']+'/wandb/latest-run/files/'+os.path.basename(config['model_settings']['model_path'])}

    # read python results from pickle file, 
    with open(config['eval_settings']['pickle_file'],'rb') as fi:
        results = pickle.load(fi)  

    # Write out run summary to results tracking document

    results_dict =  {'run_str': run_str,
                                        # 'wandb_id':results['wandb_id'],
                                        'num_ids':num_ids,
                                        'num_images_per_id':num_images,
                                        'total_training_images':len(pd.read_csv(train_file))-(len(pd.read_csv(train_file))*config['data_settings']['percent_valid']),
                                        'batch_size':config['data_settings']['batch_size'],
                                        'num_epochs':config['train_settings']['num_epochs'],
                                        # 'train_loss':results['train_loss'],
                                        # 'valid_loss':results['valid_loss'],
                                        '1NN':results['1NN_acc'],
                                        '3NN':results['3NN_acc'],
                                        'training_file':train_file,
                                        'query_file':config['data_settings']["datafiles"]["test"],
                                        # 'start_time':results['start_time'],
                                        # 'train_time':results['train_time'],
                                        # 'stop_epoch':results['stop_epoch']}
                    }

    if not os.path.exists(config['eval_settings']['results_file']):
        !touch {config['eval_settings']['results_file']}
        results_df = pd.DataFrame(results_dict,index=[0])
        results_df.to_csv(config['eval_settings']['results_file'])
    else:
        #read df and append row
        results_df = pd.read_csv(config['eval_settings']['results_file'])
        results_df.loc[len(results_df)] = results_dict
        results_df.to_csv(config['eval_settings']['results_file'],index=False)

        
        
    
    

1400
Date and time when this experiment was started: 24-09-18 12:29
Data Settings:
{'aug_p': 0.3, 'batch_size': 64, 'crop_height': None, 'crop_left': None, 'crop_top': None, 'crop_width': None, 'cropped': False, 'datafiles': {'gallery': '/home/lmeyers/ReID_complete/summer_2023_reid_galleries_closed.csv', 'query': '/home/lmeyers/summer_bee_data_reextract/new_open_max_ids_batch2/summer_bee_dataset_open_train_bee_64_ids_batch2_sample_num_max.csv', 'reference': '/home/lmeyers/summer_bee_data_reextract/new_open_max_ids_batch2/summer_bee_dataset_open_train_bee_64_ids_batch2_sample_num_02.csv', 'test': '/home/lmeyers/summer_bee_data_reextract/new_open_max_ids_batch2/summer_bee_dataset_open_train_bee_64_ids_batch2_sample_num_max.csv', 'train': '/home/lmeyers/summer_bee_data_reextract/new_open_max_ids_batch1/summer_bee_dataset_open_train_bee_64_ids_batch1_sample_num_64.csv', 'valid': ''}, 'dataset': 'summer_2023', 'fname_col': 'new_filepath', 'gallery_id': 'gallery_id', 'image_id_col': 'image_i

/home/lmeyers/anaconda3/envs/mlenv/lib/python3.9/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


Found device: cuda
Evaluating model...
Using 2173 samples for reference set
8693 total test samples
generating embeddings
Reference (or Train) Loss: 0.2061
Reference size: (2173, 128)
Test (or Query) Loss: 0.2015
Test (or Query) size: (8693, 128)
Training kNN classifier with k=1
1NN test accuracy: 0.0276
Training kNN classifier with k=3
3NN test accuracy: 0.0298

Per label 3NN test accuracy:
65	0.09
66	0.00
67	0.03
68	0.02
69	0.40
70	0.00
71	0.00
72	0.00
73	0.04
74	0.00
75	0.01
76	0.00
77	0.00
78	0.03
79	0.00
80	0.11
81	0.11
82	0.00
83	0.01
84	0.00
85	0.00
86	0.00
87	0.10
88	0.00
89	0.00
90	0.10
91	0.00
92	0.00
93	0.02
94	0.00
95	0.00
96	0.00
97	0.00
98	0.00
99	0.00
100	0.00
101	0.01
102	0.00
103	0.00
104	0.00
105	0.00
106	0.00
107	0.00
108	0.00
109	0.00
110	0.00
111	0.00
112	0.00
113	0.00
114	0.00
115	0.00
116	0.00
117	0.00
118	0.00
119	0.00
120	0.00
121	0.00
122	0.00
123	0.00
124	0.00
125	0.00
126	0.00
127	0.00
128	0.00

Printing Confusion Matrix:
[[15  1  9 ...  0  0  0]
 [ 0  0  0 

NameError: name 'num_ids' is not defined

In [7]:
 with open(yml_config['eval_settings']['pickle_file'],'rb') as fi:
        results = pickle.load(fi)  
        
    
# Convert ndarray to list

for key in results.keys():
    print(key)


1NN_acc
3NN_acc
label_list
knn_class
knn_conf
